# DAD for Wave by Triplet Loss

## Settings

In [1]:
from pathlib import Path
import sys

In [2]:
import torch
from torchvision import transforms

In [3]:
sys.path.append('../src')

import utils
from dataloader import ESC50DatasetTriplet
from models import EmbeddingNet, TripletNet
from loss_functions import TripletLoss
from trainer import fit

In [4]:
no_cuda = False
seed = 0

In [5]:
data_dir = Path().cwd().parent/'data'
raw_data_dir = data_dir/'external'/'ESC-50'
processed_data_dir = data_dir/'processed'/'ESC-50'

model_path = Path().cwd().parent/'models'/'model.pth'
embeddingnet_path = Path().cwd().parent/'models'/'embeddingnet.pth'

In [6]:
metadata_path = raw_data_dir/'meta'/'esc50.csv'
audio_dir = raw_data_dir/'audio'
spectrogram_dir = processed_data_dir/'spectrogram'

In [7]:
train_metadata_path = processed_data_dir/'metadata_train.csv'
valid_metadata_path = processed_data_dir/'metadata_valid.csv'
test_metadata_path = processed_data_dir/'metadata_test.csv'

In [8]:
use_cuda = torch.cuda.is_available() and (not no_cuda)
device = 'cuda' if use_cuda else 'cpu'
torch.manual_seed(seed)

print('device: {}'.format(device))

device: cpu


## Train

### Hyperparameters

In [9]:
input_size = (257, 431)
output_size = 128

margin = 1.

lr = 1e-5
weight_decay = 1e-6

batch_size = 2
n_epochs = 2

log_interval = 100

### Dataloaders

In [10]:
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [11]:
transform = transforms.Compose([
    transforms.ToTensor(),
    ])

In [12]:
dataset_train = ESC50DatasetTriplet(
    train_metadata_path, audio_dir, spectrogram_dir, transform,
    )

In [13]:
dataset_valid = ESC50DatasetTriplet(
    valid_metadata_path, audio_dir, spectrogram_dir, transform,
    )

In [14]:
dataloader_train = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size, shuffle=True, **kwargs)

In [15]:
dataloader_valid = torch.utils.data.DataLoader(dataset_valid, batch_size=batch_size, shuffle=False, **kwargs)

### Model

In [16]:
embedding_net = EmbeddingNet(input_size, output_size).to(device)
model = TripletNet(embedding_net).to(device)

In [17]:
loss_function = TripletLoss(margin)
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 8, gamma=0.1, last_epoch=-1)

### Train

In [ ]:
torch.backends.cudnn.benchmark = True

In [ ]:
fit(
    dataloader_train,
    dataloader_valid,
    model, loss_function, optimizer, scheduler,
    n_epochs, use_cuda, log_interval,
    )

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.
ERROR:tornado.general:Uncaught exception, closing connection.
Traceback (most recent call last):
  File "C:\Users\User\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-19-164fd9a4ba7d>", line 5, in <module>
    n_epochs, use_cuda, log_interval,
  File "../src\trainer.py", line 25, in fit
    train_loss, metrics = train_epoch(train_loader, model, loss_fn, optimizer, cuda, log_interval, metrics)
  File "../src\trainer.py", line 60, in train_epoch
    outputs = model(*data)
  File "C:\Users\User\Anaconda3\lib\site-packages\torch\nn\modules\module.py", line 477, in __call__
    result = self.forward(*input, **kwargs)
  File "../src\models.py", line 122, in forward
    embed_anc = self.embedding_net(anchor)
  File "C:\Users\User\Anaconda3\lib\site-packages\torch\nn\modules\mo

In [ ]:
torch.save(model.state_dict(), str(model_path))
torch.save(embedding_net.state_dict(), str(embeddingnet_path))

### Test

In [ ]:
model.load_state_dict(torch.load(str(model_path)))